In [1]:

import torch
import syft as sy
hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
bill = sy.VirtualWorker(hook, id="bill")

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/guoxinli/.pyenv/versions/3.7.0/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


## basic secret sharing operation

In [2]:

x = torch.tensor([25])

In [3]:
encrypted_x = x.share(bob, alice, bill)

In [4]:
encrypted_x.get()

tensor([25])

In [5]:
list(bob._tensors.values())

[]

In [6]:

x = torch.tensor([25]).share(bob, alice, bill)

In [7]:
# Bob's share
bobs_share = list(bob._tensors.values())
bobs_share

[tensor([-8064908484375400462])]

In [8]:
alices_share = list(alice._tensors.values())
alices_share

[tensor([8315361137029819119])]

In [9]:
# Bill's share
bills_share = list(bill._tensors.values())
bills_share

[tensor([-250452652654418632])]

In [10]:
-8064908484375400462+8315361137029819119-250452652654418632

25

In [2]:
x = torch.tensor([25]).share(bob,alice)
y = torch.tensor([5]).share(bob,alice)

In [3]:
z = x + y

In [7]:
bob._tensors

{65635859077: tensor([-5551104022579812131]),
 32366112700: tensor([5497649681775661102]),
 73455350153: tensor([-53454340804151029])}

In [8]:
alice._tensors

{74478689427: tensor([5551104022579812156]),
 33840861799: tensor([-5497649681775661097]),
 4946061420: tensor([53454340804151059])}

In [9]:
-53454340804151029+53454340804151059

30

## train a NN

In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/guoxinli/.pyenv/versions/3.7.0/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


In [2]:

# Set everything up
hook = sy.TorchHook(torch) 

hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
james = sy.VirtualWorker(hook, id="james")

compute_nodes = [bob, alice]

{'max_size': None, 'timeout': 60, 'url': 'ws://localhost:8777'}
{'max_size': None, 'timeout': 60, 'url': 'ws://localhost:8778'}
{'max_size': None, 'timeout': 60, 'url': 'ws://localhost:8779'}


In [4]:

# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1]])
target = torch.tensor([[0],[0],[1],[1]])

# A Toy Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 1)

    def forward(self, x):
        x = self.fc1(x)
        return x
model = Net()

In [5]:
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

remote_dataset = ([(data_bob, target_bob)], [(data_alice, target_alice)])

def update(data, target, model, optimizer):
    model.send(data.location)
    optimizer.zero_grad()
    pred = model(data)
    loss = F.mse_loss(pred.view(-1), target)
    loss.backward()
    optimizer.step()
    return model

bobs_model = Net()
alices_model = Net()

bobs_optimizer = optim.SGD(bobs_model.parameters(), lr=0.001)
alices_optimizer = optim.SGD(alices_model.parameters(), lr=0.001)

models = [bobs_model, alices_model]
params = [list(bobs_model.parameters()), list(alices_model.parameters())]
optimizers = [bobs_optimizer, alices_optimizer]

In [6]:
for name, param in models[0].named_parameters():
    print(name, param)

fc1.weight Parameter containing:
tensor([[-0.3632, -0.4681]], requires_grad=True)
fc1.bias Parameter containing:
tensor([0.0502], requires_grad=True)


In [7]:

# this is selecting which batch to train on
data_index = 0
# update remote models
# we could iterate this multiple times before proceeding, but we're only iterating once per worker here
for remote_index in range(2):
    data, target = remote_dataset[remote_index][data_index]
    models[remote_index] = update(data, target, models[remote_index], optimizers[remote_index])

Websocket connection closed (worker: bob)
Created new websocket connection


WebSocketConnectionClosedException: Connection is already closed.

In [82]:
# encrypted aggregation
new_params = list()
for param_i in range(len(params[0])):
    spdz_params = list()
    for remote_index in range(2):
        a = params[remote_index][param_i].fix_precision().share(bob, alice, crypto_provider=james)
        spdz_params.append(params[remote_index][param_i].fix_precision().share(bob, alice, crypto_provider=james).get())
    new_param = (spdz_params[0] + spdz_params[1]).get().float_precision()/2
    new_params.append(new_param)
    
    

(Wrapper)>[PointerTensor | me:88669330293 -> bob:38614593036]
(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:3630646173 -> bob:3674206146]
	-> [PointerTensor | me:41951546679 -> alice:50926425153]
	*crypto provider: james*
(Wrapper)>[PointerTensor | me:41220742481 -> alice:45221804450]
(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:59397771321 -> bob:34137705659]
	-> [PointerTensor | me:55388851597 -> alice:7105617833]
	*crypto provider: james*
(Wrapper)>[PointerTensor | me:50087019939 -> bob:11124011931]
(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:34224232126 -> bob:42969439563]
	-> [PointerTensor | me:20415306741 -> alice:62486085331]
	*crypto provider: james*
(Wrapper)>[PointerTensor | me:98921754070 -> alice:69695012392]
(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:24753684285 -> bob:44691047442]
	-> [PointerTensor | me:49848025256 -> alice:4527621025

In [68]:
spdz_params

[(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:13761091940 -> bob:28043645217]
 	-> [PointerTensor | me:59489665443 -> alice:61127392054]
 	*crypto provider: james*,
 (Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:6663001469 -> bob:2819426861]
 	-> [PointerTensor | me:38744855311 -> alice:40958496799]
 	*crypto provider: james*]

In [57]:
# cleanup
with torch.no_grad():
    for model in params:
        for param in model:
            print(param)
            param *= 0

    for model in models:
        model.get()

    for remote_index in range(2):
        for param_index in range(len(params[remote_index])):
            params[remote_index][param_index].set_(new_params[param_index])

(Wrapper)>[PointerTensor | me:50370583883 -> bob:95879494503]
(Wrapper)>[PointerTensor | me:55129873748 -> bob:22100396112]
(Wrapper)>[PointerTensor | me:35987635156 -> alice:29577394443]
(Wrapper)>[PointerTensor | me:64369435988 -> alice:46323564574]


In [58]:
for name, param in models[0].named_parameters():
    print(name, param)

fc1.weight Parameter containing:
tensor([[-0.3280,  0.1880]], requires_grad=True)
fc1.bias Parameter containing:
tensor([-0.1080], requires_grad=True)


In [84]:
params[0]

[Parameter containing:
 (Wrapper)>[PointerTensor | me:54090373536 -> bob:55179505112],
 Parameter containing:
 (Wrapper)>[PointerTensor | me:43237417535 -> bob:34757978036]]